In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(data, target)

#### 검증 세트 
    - 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법으로 훈련 세트에서 다시 떼어 낸 데이터 세트
    - 전체 데이터에서 훈련세트를 80%, 테스트세트를 20%로 나눴다면 훈련세트 80%에서 20%를 검증세트로 나누는것
    - 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가

In [4]:
sub_input, val_input, sub_target, val_target = \
train_test_split(train_input, train_target) 
# 훈련세트 train_input, train_target에서 훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target으로 나누기

In [5]:
sub_input.shape # 훈련세트

(3654, 3)

In [6]:
val_input.shape # 검증세트

(1218, 3)

In [7]:
test_input.shape # 테스트 세트

(1625, 3)

In [8]:
# 결정트리 이용
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
print(dt.score(test_input, test_target)) # 훈련 세트에 과대 적합

0.9978106185002736
0.8645320197044335
0.8529230769230769


#### 교차 검증
    - 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하고, 이 점수를 평균하여 최종 검증 점수를 얻음
    - 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련
    - 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법

#### cross_validate = 교차 검증 함수
    - 첫 번째 매개변수에 교차 검증을 수행할 모델 객체를 전달, 두번째와 세번째 매개변수에 특성과 타깃 데이터를 전달
    - n_jobs 매개변수는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정, 기본값은 1로 하나의 코어를 사용, -1로 지정하면 모든 코어를 사용
    - return_train_score 매개변수를 True로 지정하면 훈련 세트의 점수도 반환

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) 
# 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달, 그다음 앞에서처럼 직접 검증 세트를 떼어 내지 않고 훈련세트 전체를 cross_validate()함수에 전달
print(scores) # 5번의 교차 검증
# 'fit_time', 'score_time' : 각각 모델을 훈련하는 시간과 검증하는 시간
# 'test_score' : 기본적으로 5번의 교차 검증을 수행

{'fit_time': array([0.00455546, 0.00452948, 0.00474453, 0.00556493, 0.00440145]), 'score_time': array([0.00197887, 0.00224876, 0.00122499, 0.00162125, 0.00123   ]), 'test_score': array([0.84410256, 0.84717949, 0.85010267, 0.85626283, 0.84394251])}


In [10]:
# 교차 검증의 최종 점수
import numpy as np
print(np.mean(scores['test_score']))


0.8483180118991207


#### 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터
#### 하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야하만 하는 파라미터, 모두 클래스나 메서드의 매개변수로 표현
#### grid search (그리드 서치) : 매개변수가 여러개일때 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행, 별도로 cross_validate()함수를 호출 하지 않아도 됨
                                 - 하이퍼 파라미터 탐색을 자동화해주는 도구 ,
                                 - 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택,
                                 - 마지막으로 이 매개변수 조합으로 최종 모델을 훈련

In [11]:
# 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기
from sklearn.model_selection import GridSearchCV
# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들기
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004]
    
}

In [24]:
gs = GridSearchCV(dt, params, n_jobs = -1) 
# n_jobs의 기본값은 1, -1로 설정하면 시스템에 있는 모든 코어를 사용
# 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달
# 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달

In [25]:
gs.fit(train_input, train_target) # 'min_impurity_decrease'값을 바꿔가며 총 5번 실행

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004]})

In [27]:
gs.best_estimator_ # 내가 계산한 값중에서 어떤 결과가 가장 좋았어 # 0.0003를 넣었을 떄 결과가 가장 좋았음

DecisionTreeClassifier(min_impurity_decrease=0.0003)

In [28]:
dt = gs.best_estimator_ 
print(dt.score(train_input, train_target))


0.9088669950738916
